In [2]:
from fastai.vision.all import *

model_paths = [
    "resnet34", 
    "resnet101",
    "efficientnet_b3",
    "efficientnet_b3_rmce",
]

learners = []

In [3]:
for model_path in model_paths:
    learn = load_learner("model/" + model_path + "/model.pkl")
    learners.append(learn)

def ensemble_predict(items):
    preds_all = []
    for learn in learners:
        preds = learn.get_preds(dl=learn.dls.test_dl(items))[0]  # get_preds returns tuple, first is preds
        preds_all.append(preds)
    
    # Stack predictions: shape = (num_models, num_items)
    preds_stack = torch.stack(preds_all)
    
    # Average across models (dim=0)
    preds_mean = preds_stack.mean(dim=0)
    return preds_mean



/opt/anaconda3/lib/python3.10/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


In [4]:
pred = ensemble_predict('images/1_image_1.jpg')
pred

tensor([[10.9447]])

In [5]:
from fastai.vision.all import PILImage, show_image
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("data/image_labels.csv")

errors = []

pbar = tqdm(df.iterrows(), total=len(df), desc="Processing images", leave=True)

for i, row in pbar:
    try:
        img_path = f"{row['filename']}"  # change if your image path differs
        probs = ensemble_predict(img_path)
        pred = float(probs[0])
        actual = float(row['target'])
        error = abs(pred - actual)

        errors.append({
            "img_path": img_path,
            "img": img,
            "actual": actual,
            "pred": pred,
            "error": error
        })

        # Optionally update progress bar postfix
        pbar.set_postfix({"last_error": f"{error:.2f}"})

    except Exception as e:
        pbar.write(f"Skipping {row['filename']} due to error: {e}")
        
# Sort and show top 10 worst predictions
top_errors = sorted(errors, key=lambda x: x['error'], reverse=True)[:10]

for idx, item in enumerate(top_errors):
    print(f"\n#{idx+1}")
    print(f"File: {item['img_path']}")
    print(f"Predicted: {item['pred']:.2f}")
    print(f"Actual: {item['actual']:.2f}")
    print(f"Error: {item['error']:.2f}")
    show_image(item['img'], title=f"#{idx+1}: Error {item['error']:.2f}")


Processing images:   0%|                                                                                       | 0/1587 [00:00<?, ?it/s]

Processing images:   0%|                                                                                       | 0/1587 [00:00<?, ?it/s]

Skipping 0_image_1.jpg due to error: [Errno 2] No such file or directory: '/Users/kalanjarvis-loewen/Desktop/Coding/learning/ml/LeanAI/core/0_image_1.jpg'


Processing images:   0%|                                                                                       | 0/1587 [00:00<?, ?it/s]

Skipping 0_image_2.jpg due to error: [Errno 2] No such file or directory: '/Users/kalanjarvis-loewen/Desktop/Coding/learning/ml/LeanAI/core/0_image_2.jpg'


...

Processing images:  81%|████████████████████████████████████████████████████████████▋              | 1283/1587 [00:04<00:01, 223.31it/s]

Skipping 661_image_1.jpg due to error: [Errno 2] No such file or directory: '/Users/kalanjarvis-loewen/Desktop/Coding/learning/ml/LeanAI/core/661_image_1.jpg'


Processing images:  81%|████████████████████████████████████████████████████████████▋              | 1283/1587 [00:04<00:01, 223.31it/s]

Skipping 661_image_2.jpg due to error: [Errno 2] No such file or directory: '/Users/kalanjarvis-loewen/Desktop/Coding/learning/ml/LeanAI/core/661_image_2.jpg'


IOPub message rate exceeded.                                                                                                            
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [7]:
import io
from ipywidgets import widgets, VBox, HBox
from IPython.display import display

def pil_to_bytes(img):
    buf = io.BytesIO()
    img.save(buf, format='PNG')
    return buf.getvalue()

rows = []
for idx, item in enumerate(top_errors):
    img_bytes = pil_to_bytes(item['img'])
    img_widget = widgets.Image(value=img_bytes, format='png', width=200, height=200)
    
    text_html = f"""
    <b>#{idx+1}</b><br>
    File: {item['img_path']}<br>
    Predicted: {item['pred']:.2f}<br>
    Actual: {item['actual']:.2f}<br>
    Error: {item['error']:.2f}
    """
    text_widget = widgets.HTML(value=text_html)
    
    row = HBox([img_widget, text_widget])
    rows.append(row)

display(VBox(rows))


VBox()